# Problem-1


In [ ]:
!pip install -q sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

text = "My pet is Cherry"

embedding = model.encode(text)

print("Embedding Vector:")
print(f"First 5 dimensions: {embedding[:5]}")

print("\nEmbedding Length:")
print(len(embedding))


Embedding Vector:
First 5 dimensions: [ 0.05304929 -0.03090416  0.0865428   0.02706999 -0.07951166]

Embedding Length:
384


### *Problem-2*

In [17]:
!pip install -qU langchain langchain-pinecone langchain-openai
!pip install -qU google-genai
!pip install -qU langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 19.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.47.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 2.8 MB/s eta 0:00:00


In [18]:
from google.colab import userdata
from pinecone import Pinecone
from pinecone import ServerlessSpec
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document
from langchain_pinecone import PineconeVectorStore
GEMINI_API_KEY=userdata.get("GEMINI_API_KEY")
embeddings = GoogleGenerativeAIEmbeddings(
    api_key=GEMINI_API_KEY,
    model="gemini-embedding-001"
)
PINECONE_API_KEY=userdata.get("pincone_api_key")
index_name = "rag-demo-index"

pc = Pinecone(api_key=PINECONE_API_KEY)
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=3072,  # must match embedding model
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

In [19]:
doc1 = Document(page_content="Shah Rukh Khan, known as the King of Bollywood, is celebrated for his romantic roles and strong screen presence. He has starred in numerous iconic films like Dilwale Dulhania Le Jayenge and Chennai Express.")
doc2 = Document(page_content="Aamir Khan is known for his perfectionism and thought-provoking films. He has delivered memorable performances in movies like Lagaan, Dangal, and Taare Zameen Par, often focusing on social issues.")
doc3 = Document(page_content="Salman Khan is famous for his action-packed roles and mass appeal. Known for movies like Bajrangi Bhaijaan and Dabangg, he also hosts the popular reality show Bigg Boss.")
doc4 = Document(page_content="Ranbir Kapoor is admired for his versatility and charm. With films like Rockstar, Barfi, and Sanju, he has earned critical acclaim and a huge fan following among the youth.")
doc5 = Document(page_content="Deepika Padukone is one of the most successful actresses in Bollywood. Known for her powerful performances in films like Padmaavat and Piku, she has also worked in international projects.")

docs = [doc1, doc2, doc3, doc4, doc5]
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [20]:
vector_store.add_documents(docs)

['2db5e776-52fa-4bbc-9af2-a7c9f6d0bebe',
 'd5484911-b838-424c-9c3d-beb192c919d5',
 'ebc3e03d-9b90-4823-8b4e-5ff3ff2de77f',
 '55ec0dc2-81d4-4841-a1a9-bbca301ef664',
 'a3c320d3-66f0-4dc7-8877-e6d24cf9b3ed']

In [21]:
results = vector_store.similarity_search(
    "Which actor is known for fighting movies",
    k=2
)

for doc in results:
    print(doc.page_content)
    print()

Salman Khan is famous for his action-packed roles and mass appeal. Known for movies like Bajrangi Bhaijaan and Dabangg, he also hosts the popular reality show Bigg Boss.

Aamir Khan is known for his perfectionism and thought-provoking films. He has delivered memorable performances in movies like Lagaan, Dangal, and Taare Zameen Par, often focusing on social issues.

